# Memory Consumption in Python

There are two approaches mainly used for calculating the memory consumption:

In [1]:
from pympler.tracker import SummaryTracker
import linecache

import tracemalloc

Define a helper function to reduce unnecessary information:

In [2]:
def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [3]:
tracker = SummaryTracker()
tracemalloc.start()

aa = dict()
for jj in [ii for ii in range(100000)]:
    aa[jj] = jj

print("Memory Consumption (1):")
tracker.print_diff()

print("\nMemory Consumption (2):")
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)


Memory Consumption (1):
                           types |   # objects |   total size
================================ | =========== | ============
                            dict |           3 |      5.00 MB
                            list |        9302 |    802.55 KB
                             str |        9306 |    667.57 KB
                             int |        2154 |     58.90 KB
               method_descriptor |          13 |    936     B
                            code |           2 |    422     B
                         weakref |           2 |    144     B
           function (store_info) |           1 |    136     B
  function (_schedule_in_thread) |           1 |    136     B
              wrapper_descriptor |           1 |     72     B
               member_descriptor |           1 |     64     B
                            cell |           1 |     40     B
                  sqlite3.Cursor |          -1 |   -112     B
                   list_iterator |          -3